In [2]:
from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.analysis.diffraction import xrd
from pymatgen.core.lattice import Lattice
from pymatgen import Structure
from jupyter_jsmol.pymatgen import quick_view

In [3]:
# Récupération de la structure
with MPRester("oZkH8kj10DOmI5dC") as m:
    
    structure = m.get_structure_by_material_id("mp-9124")
    
    # Détermination du type de maille et du système cristallin
    analyzer = SpacegroupAnalyzer(structure)
    print("Type de maille :", analyzer.get_lattice_type())
    print("Système cristallin :", analyzer.get_crystal_system())

    # Détermination du groupe ponctuel
    print("Groupe ponctuel :", analyzer.get_point_group_symbol())

    # Détermination du groupe ponctuel
    print("Groupe d'espace :", analyzer.get_space_group_symbol())
    
    symops = SpacegroupAnalyzer(structure).get_symmetry_operations()
    
    
    print(structure)

Type de maille : cubic
Système cristallin : cubic
Groupe ponctuel : -43m
Groupe d'espace : F-43m
Full Formula (Li1 Zn1 As1)
Reduced Formula: LiZnAs
abc   :   4.227421   4.227421   4.227421
angles:  60.000000  60.000000  60.000000
Sites (3)
  #  SP       a     b     c    magmom
---  ----  ----  ----  ----  --------
  0  Li    0.75  0.75  0.75         0
  1  Zn    0.25  0.25  0.25        -0
  2  As    0     0     0           -0


In [4]:
view = quick_view(structure)
display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [5]:
Li = [.75, .75, .75]
Zn = [.25, .25, .25]
As = [  0,   0,   0]

#symops: id, name, atomno
symmetries = [(7, 'C2 axis (y, x, z-x-y)', 2),(9, 'C3 axis (-x-y-z, x, z)', 1),(14, 'Mirror plane (y, x, z)', 3)]

LiSym = [symops[sid].operate(Li) for sid, _, aid in symmetries if aid == 1][0]
ZnSym = [symops[sid].operate(Zn) for sid, _, aid in symmetries if aid == 2][0]
AsSym = [symops[sid].operate(As) for sid, _, aid in symmetries if aid == 3][0]

In [6]:
symid1, symname1, atomno1 = [(a, b, c) for a, b, c in symmetries if c == 1][0]
print(f'Symmetry: {symname1}')
print(f'Li {Li} -> {LiSym}')

view1 = quick_view(structure)
display(view1)

Symmetry: C3 axis (-x-y-z, x, z)
Li [0.75, 0.75, 0.75] -> [ 0.75  0.75 -2.25]


JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [7]:
view1.script(f'draw symop {symid1} {{atomno={atomno1}}}')

In [8]:
symid2, symname2, atomno2 = [(a, b, c) for a, b, c in symmetries if c == 2][0]
print(f'Symmetry: {symname2}')
print(f'Zn {Zn} -> {ZnSym}')

view2 = quick_view(structure)
display(view2)

Symmetry: C2 axis (y, x, z-x-y)
Zn [0.25, 0.25, 0.25] -> [0.25 0.25 0.25]


JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [9]:
view2.script(f'draw symop {symid2} {{atomno={atomno2}}}')

In [10]:
symid3, symname3, atomno3 = [(a, b, c) for a, b, c in symmetries if c == 3][0]
print(f'Symmetry: {symname3}')
print(f'As {As} -> {AsSym}')

view3 = quick_view(structure)
display(view3)

Symmetry: Mirror plane (y, x, z)
As [0, 0, 0] -> [0. 0. 0.]


JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [11]:
view3.script(f'draw symop {symid3} {{atomno={atomno3}}}')